In [1]:
import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit, fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [12]:
basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/P.h5'
with h5py.File(basePath) as f:
    cfgList=list(f.keys())
basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/pi0DoubleLoop.h5'
with h5py.File(basePath) as f:
    cfgList=list(set(cfgList)&set(f.keys()))

Ncfg=len(cfgList)
print(Ncfg)

data={}

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/P.h5'
with h5py.File(basePath) as f:
    momList=f['0000/mvec'][()]
    # for ky in ['P0DN','P0UP','PPDN','PPUP']:
    #     data[ky]=np.array([f[cfg][ky][:,:,0,0]+1j*f[cfg][ky][:,:,0,1] for cfg in cfgList])

# basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/pi0DoubleLoop.h5'
# with h5py.File(basePath) as f:
#     for ky in ['pi0DoubleLoop']:
#         data[ky]=np.array([f[cfg][ky] for cfg in cfgList])

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/pi0Loop.h5'
with h5py.File(basePath) as f:
    for ky in ['pi0Loop','pi0Loop_g5_vev']:
        data[ky]=np.array([f[cfg][ky][()] for cfg in cfgList])

momDic={}
for i in range(len(momList)):
    momDic[tuple(momList[i])]=i

t_total=48

311


In [ ]:
i_mom=momDic[(0,0,0)]
dat=np.array([[data['pi0Loop'][cfg,:,i_mom,0],data['pi0Loop_g5_vev'][cfg]] for cfg in range(len(cfgList))],dtype=object)

def tFunc(dat):
    t=np.mean(dat,axis=0)
    tDat=np.vstack(dat[:,0])
    tDat=tDat-t[1]

    t2pt=np.transpose([np.mean(np.roll(tDat,-tf,axis=1)*tDat,axis=1) for tf in range(t_total)])

    # t=tDat[0]*tDat[0]*tDat*tDat
    # t=np.mean(t,axis=0)
    # tSub=tDat[0]*tDat
    # tSub=np.mean(tSub,axis=0)

    shift=24

    t=np.roll(t2pt,-shift,axis=1)*t2pt
    t=np.mean(t,axis=0)
    tSub=t2pt
    tSub=np.mean(tSub,axis=0)

    return (t-np.roll(tSub,-shift,axis=0)*tSub)/len(dat)

(mean,cov,err)=yu.jackknife(dat,tFunc)
print(mean)
print()
print(err)